In [1]:
import numpy as np
import utils.data_utils as du
from utils.evaluator import binarize_label
import torch
import matplotlib.pyplot as plt
import torch.nn.functional as F


In [ ]:
data_dir = "/home/deeplearning/Abhijit/nas_drive/Abhijit/WholeBody/CT_ce/Data/Visceral"
label_dir = "/home/deeplearning/Abhijit/nas_drive/Abhijit/WholeBody/CT_ce/Data/Visceral"


support_volume, support_labelmap, _, _ = du.load_and_preprocess(support_file_paths[0],
                                                                orientation=orientation,
                                                                remap_config=remap_config)
support_volume = support_volume if len(support_volume.shape) == 4 else support_volume[:, np.newaxis, :,
                                                                       :]
support_volume, support_labelmap = torch.tensor(support_volume).type(torch.FloatTensor), \
                                   torch.tensor(support_labelmap).type(torch.LongTensor)

support_volume, range_index = binarize_label(support_volume, support_labelmap, query_label)

support_slice_indexes = np.round(np.linspace(0, len(support_volume) - 1, Num_support + 1)).astype(int)
support_slice_indexes += (len(support_volume) // Num_support) // 2
support_slice_indexes = support_slice_indexes[:-1]


Loading dataset
Train size: 2576
Test size: 304


In [283]:
class_label = 7
fold = '2'

X, y = train_data.X, train_data.y
y = (y == class_label)
y = y.astype(np.float32)

batch_size, _, _ = y.shape

slice_with_class = np.sum(y.reshape(batch_size, -1), axis=1) > 10
X = X[slice_with_class]
y = y[slice_with_class]

query_slice = np.random.randint(0, len(X))
support_slice = np.random.randint(0, len(X))


no_skip_model = torch.load('saved_models/sne_position_all_type_spatial_fold'+fold+'.pth.tar')
# skip_model = torch.load('saved_models/sne_position_all_type_spatial_skipconn_baseline_fold'+fold+'.pth.tar')

no_skip_model.cuda()
no_skip_model.eval()
# skip_model.cuda()
# skip_model.eval()

query_input = torch.tensor(X[query_slice])
query_gt = torch.tensor(y[query_slice])

support_input = torch.tensor(X[support_slice])
support_gt = torch.tensor(y[support_slice])

support_gt = support_gt.unsqueeze(0)

condition_input = torch.cat((support_input, support_gt), dim=0)

query_input = query_input.unsqueeze(0)
condition_input = condition_input.unsqueeze(0)

query_input = query_input.cuda()
condition_input = condition_input.cuda()

weights = no_skip_model.conditioner(condition_input)
out = no_skip_model.segmentor(query_input, weights)

_, segmentation_no_chhapa = torch.max(F.softmax(out, dim=1), dim=1)

# weights = skip_model.conditioner(condition_input)
# out = skip_model.segmentor(query_input, weights)

# _, segmentation_chhapa = torch.max(F.softmax(out, dim=1), dim=1)

ncols = 5
fig, ax = plt.subplots(nrows=1, ncols=ncols, figsize=(20, 10), squeeze=False)

ax[0][0].imshow(torch.squeeze(query_input), cmap='gray', vmin=0, vmax=1)
ax[0][0].set_title("Query Input", fontsize=10, color="blue")
ax[0][0].axis('off')
ax[0][1].imshow(torch.squeeze(query_gt), cmap='gray', vmin=0, vmax=1)
ax[0][1].set_title("Query GT", fontsize=10, color="blue")
ax[0][1].axis('off')
ax[0][2].imshow(torch.squeeze(segmentation_no_chhapa), cmap='gray', vmin=0, vmax=1)
ax[0][2].set_title("No Chhapa", fontsize=10, color="blue")
ax[0][2].axis('off')
# ax[0][3].imshow(torch.squeeze(segmentation_chhapa), cmap='gray', vmin=0, vmax=1)
# ax[0][3].set_title("Chhapa", fontsize=10, color="blue")
# ax[0][3].axis('off')
ax[0][3].imshow(torch.squeeze(support_input), cmap='gray', vmin=0, vmax=1)
ax[0][3].set_title("Support Input", fontsize=10, color="blue")
ax[0][3].axis('off')
ax[0][4].imshow(torch.squeeze(support_gt), cmap='gray', vmin=0, vmax=1)
ax[0][4].set_title("Support GT", fontsize=10, color="blue")
ax[0][4].axis('off')

fig.set_tight_layout(True)
plt.show()
